# Import Module

In [1]:
# import module
import os
import glob
import time
import math
import numpy as np
import pandas as pd
import scipy.io as sio
from IPython.display import clear_output

import torch
from torch import optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
# for loss function
from typing import Optional
#from kornia.utils.one_hot import one_hot

from models import DP_UNet, DP_resUNet
from patch_function import makepatches_overlay, \
makepatches_overlay_normap, \
patchesback_overlay, \
find_ksp_andpadding, \
pad_back, \
pad_for_256, \
pad_back_256
from data_preprocess_and_loader import Dataset, Dataset_val
from losses_unet3d import DiceLoss, GeneralizedDiceLoss, compute_per_channel_dice
from train_and_val import train, validation

print(torch.cuda.is_available())
print(torch.__version__)

True
2.1.0


# Define Datasets

In [2]:
#local
train_image_path = 'path that save the training data'
val_image_path = 'path that save the validation data'

k = os.listdir(train_image_path)
list_tra = []
for j in k:
    list_tra.append(train_image_path + '/' + j)
list_val = []
k = os.listdir(val_image_path)
for j in k:
    list_val.append(val_image_path + '/' + j)

In [3]:
# 設置dataloader
#biparametric = True
train_data = Dataset(list_tra, biparametric=True)
train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=1,
    num_workers=12,
    pin_memory=True,
    shuffle = True)

val_data = Dataset_val(list_val, biparametric=True)
val_loader = torch.utils.data.DataLoader(
    val_data,           
    batch_size=1,          
    num_workers=12,
    pin_memory=True,
    shuffle = False)

# Initialize the Model

In [4]:
# Device
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialization

model = DP_resUNet(img_channels = 2, n_classes = 1).cuda()
#model= nn.DataParallel(model)
#model.to(device)

#model = DP_UNet(img_channels = 2, n_classes = 2).cuda()

# define optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, gamma=0.95)

# loss function
criterion_DICE = DiceLoss()
criterion_GDL = GeneralizedDiceLoss()

loss_fun = (criterion_DICE,criterion_GDL,compute_per_channel_dice)

# Train the Model

In [5]:
# Train the Model
epochs = 2 # The number of epochs

# 儲存權重的路徑
path = 'directory to save model'

if os.path.isdir(path)==False:
    os.mkdir(path)
path = os.path.join(path,"train")
if os.path.isdir(path)==False:
    os.mkdir(path)       
filenum = glob.glob(path + "/exp*")
path = path + "/exp" + str(len(filenum)+1)
os.mkdir(path)

#f = open(path + '/loss_curve.txt', 'a')
# 開始訓練
traloss = 0
valloss = 0
rvalloss = 0
for epoch in range(1,epochs+1):
    if epoch==1:
        print('Training start.')
    
    epoch_start_time = time.time()
    tralossnew = train(train_loader,model,loss_fun,optimizer,epoch,path)
    print('-' * 89)
    vallossnew = validation(val_loader,model,loss_fun,epoch,path)
    print('-' * 89)    
    print('| end of epoch {:3d} | time: {:5.2f}s'.format(epoch, (time.time() - epoch_start_time)))
    print('-' * 89)
    
    #scheduler.step()
    # save best model parameters
    f2 = open(path + '/model_info.txt', 'a')
    if tralossnew<traloss or epoch ==1: 
        fname = path + '/best_tra'  + '.tar'
        torch.save(model.state_dict(), fname)
        traloss = tralossnew
        f2.write('| best_tra | epoch {:3d}| '.format(epoch)+'\r\n')
    if vallossnew<valloss or epoch ==1: 
        fname = path + '/best_val'  + '.tar'
        torch.save(model.state_dict(), fname)
        valloss = vallossnew
        f2.write('| best_val | epoch {:3d}| '.format(epoch)+'\r\n')
        

fname = path + '/last' + '.tar'
torch.save(model.state_dict(), fname)
f2.write('Finished!')
    
#f.close()
f2.close()
clear_output(wait=True)

Training start.
| epoch   1 |   300/ 2486 batches |  ms/batch 88001.13 |  loss_dice  0.99  |  loss_gdl  0.99  |  dice_score  0.01  | 
| epoch   1 |   600/ 2486 batches |  ms/batch 86611.29 |  loss_dice  0.98  |  loss_gdl  0.98  |  dice_score  0.02  | 
| epoch   1 |   900/ 2486 batches |  ms/batch 97571.58 |  loss_dice  0.98  |  loss_gdl  0.98  |  dice_score  0.02  | 
| epoch   1 |  1200/ 2486 batches |  ms/batch 111347.59 |  loss_dice  0.96  |  loss_gdl  0.96  |  dice_score  0.04  | 
| epoch   1 |  1500/ 2486 batches |  ms/batch 112887.10 |  loss_dice  0.95  |  loss_gdl  0.95  |  dice_score  0.05  | 
| epoch   1 |  1800/ 2486 batches |  ms/batch 125431.01 |  loss_dice  0.93  |  loss_gdl  0.93  |  dice_score  0.07  | 
| epoch   1 |  2100/ 2486 batches |  ms/batch 129051.31 |  loss_dice  0.90  |  loss_gdl  0.90  |  dice_score  0.10  | 
| epoch   1 |  2400/ 2486 batches |  ms/batch 118991.55 |  loss_dice  0.87  |  loss_gdl  0.87  |  dice_score  0.13  | 
| epoch   1 |  2486/ 2486 batches |

| epoch  34 |   400/  440 batches |  ms/batch 15316.35 |  loss_dice  0.33  |  loss_gdl  0.33  |  dice_score  0.67  | 
| epoch  34 |   440/  440 batches |  ms/batch 8908.13 |  loss_dice  0.34  |  loss_gdl  0.34  |  dice_score  0.66  | 
-----------------------------------------------------------------------------------------
| end of epoch  34 | time: 1295.32s
-----------------------------------------------------------------------------------------
| epoch  35 |   300/ 3762 batches |  ms/batch 104822.42 |  loss_dice  0.33  |  loss_gdl  0.33  |  dice_score  0.67  | 
| epoch  35 |   600/ 3762 batches |  ms/batch 83126.29 |  loss_dice  0.33  |  loss_gdl  0.33  |  dice_score  0.67  | 
| epoch  35 |   900/ 3762 batches |  ms/batch 82656.25 |  loss_dice  0.32  |  loss_gdl  0.32  |  dice_score  0.68  | 
| epoch  35 |  1200/ 3762 batches |  ms/batch 87733.59 |  loss_dice  0.33  |  loss_gdl  0.33  |  dice_score  0.67  | 
| epoch  35 |  1500/ 3762 batches |  ms/batch 85836.53 |  loss_dice  0.33  |

| epoch  38 |  1500/ 3762 batches |  ms/batch 94341.38 |  loss_dice  0.34  |  loss_gdl  0.34  |  dice_score  0.66  | 
| epoch  38 |  1800/ 3762 batches |  ms/batch 92491.17 |  loss_dice  0.34  |  loss_gdl  0.34  |  dice_score  0.66  | 
| epoch  38 |  2100/ 3762 batches |  ms/batch 95852.79 |  loss_dice  0.34  |  loss_gdl  0.34  |  dice_score  0.66  | 
| epoch  38 |  2400/ 3762 batches |  ms/batch 100507.65 |  loss_dice  0.34  |  loss_gdl  0.34  |  dice_score  0.66  | 
| epoch  38 |  2700/ 3762 batches |  ms/batch 93005.17 |  loss_dice  0.34  |  loss_gdl  0.34  |  dice_score  0.66  | 
| epoch  38 |  3000/ 3762 batches |  ms/batch 94636.37 |  loss_dice  0.34  |  loss_gdl  0.34  |  dice_score  0.66  | 
| epoch  38 |  3300/ 3762 batches |  ms/batch 94984.39 |  loss_dice  0.34  |  loss_gdl  0.34  |  dice_score  0.66  | 
| epoch  38 |  3600/ 3762 batches |  ms/batch 98632.31 |  loss_dice  0.34  |  loss_gdl  0.34  |  dice_score  0.66  | 
| epoch  38 |  3762/ 3762 batches |  ms/batch 48692.14 

| epoch  41 |  3762/ 3762 batches |  ms/batch 50047.83 |  loss_dice  0.34  |  loss_gdl  0.34  |  dice_score  0.66  | 
-----------------------------------------------------------------------------------------
| epoch  41 |   100/  440 batches |  ms/batch 23167.20 |  loss_dice  0.27  |  loss_gdl  0.27  |  dice_score  0.73  | 
| epoch  41 |   200/  440 batches |  ms/batch 21850.47 |  loss_dice  0.29  |  loss_gdl  0.29  |  dice_score  0.71  | 
| epoch  41 |   300/  440 batches |  ms/batch 49191.32 |  loss_dice  0.34  |  loss_gdl  0.34  |  dice_score  0.66  | 
| epoch  41 |   400/  440 batches |  ms/batch 14865.94 |  loss_dice  0.33  |  loss_gdl  0.33  |  dice_score  0.67  | 
| epoch  41 |   440/  440 batches |  ms/batch 8532.71 |  loss_dice  0.33  |  loss_gdl  0.33  |  dice_score  0.67  | 
-----------------------------------------------------------------------------------------
| end of epoch  41 | time: 1295.14s
-----------------------------------------------------------------------------

| epoch  45 |   300/ 3762 batches |  ms/batch 107425.81 |  loss_dice  0.31  |  loss_gdl  0.31  |  dice_score  0.69  | 
| epoch  45 |   600/ 3762 batches |  ms/batch 82757.71 |  loss_dice  0.31  |  loss_gdl  0.31  |  dice_score  0.69  | 
| epoch  45 |   900/ 3762 batches |  ms/batch 88497.63 |  loss_dice  0.33  |  loss_gdl  0.33  |  dice_score  0.67  | 
| epoch  45 |  1200/ 3762 batches |  ms/batch 88264.74 |  loss_dice  0.32  |  loss_gdl  0.32  |  dice_score  0.68  | 
| epoch  45 |  1500/ 3762 batches |  ms/batch 89722.92 |  loss_dice  0.32  |  loss_gdl  0.32  |  dice_score  0.68  | 
| epoch  45 |  1800/ 3762 batches |  ms/batch 96177.35 |  loss_dice  0.32  |  loss_gdl  0.32  |  dice_score  0.68  | 
| epoch  45 |  2100/ 3762 batches |  ms/batch 99304.00 |  loss_dice  0.33  |  loss_gdl  0.33  |  dice_score  0.67  | 
| epoch  45 |  2400/ 3762 batches |  ms/batch 93368.20 |  loss_dice  0.32  |  loss_gdl  0.32  |  dice_score  0.68  | 
| epoch  45 |  2700/ 3762 batches |  ms/batch 108124.99

...

| epoch 187 |  1200/ 3762 batches |  ms/batch 88374.33 |  loss_dice  0.31  |  loss_gdl  0.31  |  dice_score  0.69  | 
| epoch 187 |  1500/ 3762 batches |  ms/batch 90190.08 |  loss_dice  0.31  |  loss_gdl  0.31  |  dice_score  0.69  | 
| epoch 187 |  1800/ 3762 batches |  ms/batch 95103.03 |  loss_dice  0.31  |  loss_gdl  0.31  |  dice_score  0.69  | 
| epoch 187 |  2100/ 3762 batches |  ms/batch 97847.41 |  loss_dice  0.31  |  loss_gdl  0.31  |  dice_score  0.69  | 
| epoch 187 |  2400/ 3762 batches |  ms/batch 97468.01 |  loss_dice  0.30  |  loss_gdl  0.30  |  dice_score  0.70  | 
| epoch 187 |  2700/ 3762 batches |  ms/batch 97660.19 |  loss_dice  0.30  |  loss_gdl  0.30  |  dice_score  0.70  | 
| epoch 187 |  3000/ 3762 batches |  ms/batch 98112.00 |  loss_dice  0.30  |  loss_gdl  0.30  |  dice_score  0.70  | 
| epoch 187 |  3300/ 3762 batches |  ms/batch 103893.41 |  loss_dice  0.30  |  loss_gdl  0.30  |  dice_score  0.70  | 
| epoch 187 |  3600/ 3762 batches |  ms/batch 92290.03 

| epoch 190 |  3600/ 3762 batches |  ms/batch 105994.10 |  loss_dice  0.29  |  loss_gdl  0.29  |  dice_score  0.71  | 
| epoch 190 |  3762/ 3762 batches |  ms/batch 55209.68 |  loss_dice  0.30  |  loss_gdl  0.30  |  dice_score  0.70  | 
-----------------------------------------------------------------------------------------
| epoch 190 |   100/  440 batches |  ms/batch 24097.53 |  loss_dice  0.21  |  loss_gdl  0.21  |  dice_score  0.79  | 
| epoch 190 |   200/  440 batches |  ms/batch 23456.52 |  loss_dice  0.26  |  loss_gdl  0.26  |  dice_score  0.74  | 
| epoch 190 |   300/  440 batches |  ms/batch 52068.59 |  loss_dice  0.34  |  loss_gdl  0.34  |  dice_score  0.66  | 
| epoch 190 |   400/  440 batches |  ms/batch 16610.88 |  loss_dice  0.30  |  loss_gdl  0.30  |  dice_score  0.70  | 
| epoch 190 |   440/  440 batches |  ms/batch 9961.83 |  loss_dice  0.31  |  loss_gdl  0.31  |  dice_score  0.69  | 
------------------------------------------------------------------------------------

-----------------------------------------------------------------------------------------
| end of epoch 193 | time: 1267.04s
-----------------------------------------------------------------------------------------
| epoch 194 |   300/ 3762 batches |  ms/batch 110759.23 |  loss_dice  0.30  |  loss_gdl  0.30  |  dice_score  0.70  | 
| epoch 194 |   600/ 3762 batches |  ms/batch 82628.64 |  loss_dice  0.30  |  loss_gdl  0.30  |  dice_score  0.70  | 
| epoch 194 |   900/ 3762 batches |  ms/batch 85362.19 |  loss_dice  0.30  |  loss_gdl  0.30  |  dice_score  0.70  | 
| epoch 194 |  1200/ 3762 batches |  ms/batch 85234.15 |  loss_dice  0.30  |  loss_gdl  0.30  |  dice_score  0.70  | 
| epoch 194 |  1500/ 3762 batches |  ms/batch 86461.13 |  loss_dice  0.30  |  loss_gdl  0.30  |  dice_score  0.70  | 
| epoch 194 |  1800/ 3762 batches |  ms/batch 96489.01 |  loss_dice  0.30  |  loss_gdl  0.30  |  dice_score  0.70  | 
| epoch 194 |  2100/ 3762 batches |  ms/batch 90375.72 |  loss_dice  0.30  

| epoch 197 |  2100/ 3762 batches |  ms/batch 81185.14 |  loss_dice  0.31  |  loss_gdl  0.31  |  dice_score  0.69  | 
| epoch 197 |  2400/ 3762 batches |  ms/batch 80441.54 |  loss_dice  0.30  |  loss_gdl  0.30  |  dice_score  0.70  | 
| epoch 197 |  2700/ 3762 batches |  ms/batch 86985.51 |  loss_dice  0.31  |  loss_gdl  0.31  |  dice_score  0.69  | 
| epoch 197 |  3000/ 3762 batches |  ms/batch 85124.34 |  loss_dice  0.31  |  loss_gdl  0.31  |  dice_score  0.69  | 
| epoch 197 |  3300/ 3762 batches |  ms/batch 82507.08 |  loss_dice  0.31  |  loss_gdl  0.31  |  dice_score  0.69  | 
| epoch 197 |  3600/ 3762 batches |  ms/batch 85312.46 |  loss_dice  0.30  |  loss_gdl  0.30  |  dice_score  0.70  | 
| epoch 197 |  3762/ 3762 batches |  ms/batch 43422.68 |  loss_dice  0.30  |  loss_gdl  0.30  |  dice_score  0.70  | 
-----------------------------------------------------------------------------------------
| epoch 197 |   100/  440 batches |  ms/batch 23493.19 |  loss_dice  0.20  |  loss_g

| epoch 200 |   100/  440 batches |  ms/batch 21285.82 |  loss_dice  0.21  |  loss_gdl  0.21  |  dice_score  0.79  | 
| epoch 200 |   200/  440 batches |  ms/batch 22205.87 |  loss_dice  0.25  |  loss_gdl  0.25  |  dice_score  0.75  | 
| epoch 200 |   300/  440 batches |  ms/batch 45880.55 |  loss_dice  0.32  |  loss_gdl  0.32  |  dice_score  0.68  | 
| epoch 200 |   400/  440 batches |  ms/batch 14193.51 |  loss_dice  0.28  |  loss_gdl  0.28  |  dice_score  0.72  | 
| epoch 200 |   440/  440 batches |  ms/batch 6817.55 |  loss_dice  0.28  |  loss_gdl  0.28  |  dice_score  0.72  | 
-----------------------------------------------------------------------------------------
| end of epoch 200 | time: 1269.71s
-----------------------------------------------------------------------------------------
